Below is an illustrative Python example that demonstrates a simple version of Outlier Channel Splitting (OCS) on a small sample weight matrix. In OCS, channels (or rows, in this simplified case) that contain outlier values—i.e., values that exceed a given threshold—are split into two channels. The idea is to divide the weights so that the dynamic range is reduced, which in turn can lead to more effective low-bit quantization.

In this example, if a channel has any absolute value greater than a chosen threshold (here, 1.0), we split that channel into two copies where we divide the weights by 2. This simple operation maintains the overall contribution when the subsequent layer sums the outputs while reducing the per-channel magnitude.

In [1]:
import numpy as np

def outlier_channel_splitting(weights, threshold):
    """
    Splits channels with outlier values into two channels with reduced scaling.
    
    Parameters:
        weights (np.ndarray): 2D array where each row represents a "channel".
        threshold (float): The threshold above which a channel is considered to have an outlier.
        
    Returns:
        np.ndarray: New weights array with outlier channels split.
    """
    new_channels = []
    for i, channel in enumerate(weights):
        # Identify if the channel contains an outlier based on the threshold.
        if np.max(np.abs(channel)) > threshold:
            print(f"Channel {i} identified as outlier channel: {channel}")
            # Split the channel into two channels by dividing the weight values by 2.
            # When combined in the next layer (if summed), they should approximate the original channel's effect.
            split_channel = channel / 2.0
            new_channels.append(split_channel)
            new_channels.append(split_channel)
        else:
            new_channels.append(channel)
    return np.array(new_channels)

# --- Sample Data ---
# Each row represents a channel of weights.
# Here channels 0 and 2 have extreme (outlier) values.
weights = np.array([
    [0.1, 0.2, 5.0, 0.1],    # Outlier present (5.0)
    [0.5, 0.6, 0.55, 0.45],   # No outlier
    [0.2, 0.3, -4.8, 0.3]     # Outlier present (-4.8)
])

# Define threshold for detecting outliers.
threshold = 1.0

# Apply Outlier Channel Splitting.
new_weights = outlier_channel_splitting(weights, threshold)

print("Original weights shape:", weights.shape)
print("New weights shape after Outlier Channel Splitting:", new_weights.shape)
print("New weights array:\n", new_weights)


Channel 0 identified as outlier channel: [0.1 0.2 5.  0.1]
Channel 2 identified as outlier channel: [ 0.2  0.3 -4.8  0.3]
Original weights shape: (3, 4)
New weights shape after Outlier Channel Splitting: (5, 4)
New weights array:
 [[ 0.05  0.1   2.5   0.05]
 [ 0.05  0.1   2.5   0.05]
 [ 0.5   0.6   0.55  0.45]
 [ 0.1   0.15 -2.4   0.15]
 [ 0.1   0.15 -2.4   0.15]]


Explanation
The Function:

Input: A 2D numpy array weights, where each row is a channel, and a quantization sensitivity threshold.

Operation: For each channel, the code checks whether the maximum absolute value exceeds the threshold.

Splitting: If a channel exceeds the threshold, it is split into two copies. Each copy is divided by 2 so that, when recombined (e.g., through summation in the next network layer), they preserve the original activation magnitude.

Output: The function returns a new weights array with a higher channel count.

Sample Data:

The array contains 3 channels.

Channels 0 and 2 include high values (5.0 and -4.8, respectively) that flag them as outliers.

After processing, these channels are each replaced by two channels with scaled-down values, so the new output has 5 channels in total.

Results:

The printed shapes and values show the transformation from a 3-channel to a 5-channel weight tensor.

In a full neural network, such a transformation could help mitigate the negative impact of quantizing weights with large dynamic ranges.

Additional Considerations: This example is a very simplified version of OCS. In practice, the splitting might be more nuanced:

You might split and then adjust the biases or subsequent computations to ensure that the model's behavior remains consistent.

Advanced techniques might use learned scaling factors and even consider activation distributions.